In [1]:
import os
import json
import time
import requests
import urllib.parse
import urllib.request
import warnings
import geopandas as gpd
import pandas as pd
from osgeo import gdal, ogr
from shapely import wkt
import datetime
from tqdm.notebook import tqdm
# remove warnings
import multiprocessing

warnings.filterwarnings('ignore')

In [2]:
DATA_DIR = "./data/"
FIRE_FP = DATA_DIR + "fire.geojson"
TREAT_DIR = DATA_DIR + "treatment/"
TREAT_FPS = ["point.geojson", "line.geojson", "poly.geojson"]

# Get Fire Data

In [5]:
def get_fire_data():
    base_url = "https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/"
    
    start = time.perf_counter()
    print('=' * 90)
    print('handling {} '.format(base_url))
    
    # Get record extract limit
    url_string = base_url + "?f=json"
    print("\nGetting record extract limit", url_string)
    j = urllib.request.urlopen(url_string)
    js = json.load(j)
    max_records_count = int(js["maxRecordCount"])
    print(("Record extract limit: %s" % max_records_count))
    max_records_count = min(max_records_count, 800)
    
    # get count of objects
    count_query = "query?where=%20(YEAR_%20%3D%202022%20OR%20YEAR_%20%3D%209999)%20&outFields=*&returnCountOnly=true&outSR=4326&f=json"
    url_string = base_url + count_query
    print("\nGetting count of objects", url_string)
    j = urllib.request.urlopen(url_string)
    js = json.load(j)
    num_of_records = js['count']
    print(("Number of target records: %s" % num_of_records))
    
    print("\nGathering records…")
    gather_query = "query?where=%20(YEAR_%20%3D%202022%20OR%20YEAR_%20%3D%209999)%20&outFields=*&outSR=4326&f=json"
    url_string = base_url + gather_query
    resp = requests.get(url_string, verify=False)
    print(url_string)
    data = resp.json()
    
    print("\nProcessing data...")
    for feat in data['features']:
        geo = {'type':'polygon', 'coordinates':feat['geometry']['rings']}
        feat['geometry'] = geo
        feat['properties'] = feat['attributes']
        del feat['attributes']
    fires = gpd.GeoDataFrame.from_features(data['features'], crs='EPSG:4269')
    fires = fires.loc[fires['geometry'].is_valid, :]
    columns_lower = { col: col.lower() for col in fires.columns}
    fires = fires.rename(columns=columns_lower)
    fires['alarm_date'] = [datetime.datetime.fromtimestamp(ms/1000.0) for ms in fires['alarm_date']]
    fires['cont_date'] = [datetime.datetime.fromtimestamp(ms/1000.0) for ms in fires['cont_date']]
    fires = fires.set_index('objectid')
    fires = fires.to_crs(32614)
    
    end = time.perf_counter()
    print('=' * 90)
    print('Number of requests: {}'.format(1))
    print(f'Finished in {round(end - start, 2)} second(s)')
    return fires

In [6]:
fires = get_fire_data()

handling https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/ 

Getting record extract limit https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/?f=json
Record extract limit: 2000

Getting count of objects https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/query?where=%20(YEAR_%20%3D%202022%20OR%20YEAR_%20%3D%209999)%20&outFields=*&returnCountOnly=true&outSR=4326&f=json
Number of target records: 306

Gathering records…
https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/query?where=%20(YEAR_%20%3D%202022%20OR%20YEAR_%20%3D%209999)%20&outFields=*&outSR=4326&f=json

Processing data...
Number of requests: 1
Finished in 7.09 second(s)


In [41]:
start = time.perf_counter()
if not os.path.exists(FIRE_FP):
    print(f"'{FIRE_FP}' does not exist.\n    Getting Fire Data...")
    fires = get_fire_data()
    print(f"    Saving to {FIRE_FP}...")
    fires.to_file(FIRE_FP)
else:
    print(f"'{FIRE_FP}' already exists.\n    Loading Fire Data...")
    fires = gpd.read_file(FIRE_FP)
end = time.perf_counter()
print(f'\nFinished in {round(end - start, 2)} second(s)')

'./data/fire.geojson' does not exist.
    Getting Fire Data...
handling https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/ 

Getting record extract limit https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/?f=json
Record extract limit: 2000

Getting count of objects https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/query?where=%20(YEAR_%20%3D%202022%20OR%20YEAR_%20%3D%209999)%20&outFields=*&returnCountOnly=true&outSR=4326&f=json
Number of target records: 306

Gathering records…
https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/query?where=%20(YEAR_%20%3D%202022%20OR%20YEAR_%20%3D%209999)%20&outFields=*&outSR=4326&f=json

Processing data...
Number of requests: 1
Finished in 4.01 second(s)
    Saving to ./data/fire.geojson...

Finished in 5.3 second(s)


# Get Treatment Data

In [4]:
TREAT_URLS = ["https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/0", 
              #"https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/1", 
              "https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/2"]

def fetch_all_features(base_url):
    start = time.perf_counter()
    print('=' * 90)
    print('handling {} '.format(base_url))
    
    # Get record extract limit
    url_string = base_url + "?f=json"
    j = urllib.request.urlopen(url_string)
    js = json.load(j)
    max_records_count = int(js["maxRecordCount"])
    print(("Record extract limit: %s" % max_records_count))
    
    max_records_count = min(max_records_count, 800)
    
    # Get object ids of features
    fields = "*"
    where = "1=1"
    url_string = base_url + "/query?where={}&returnIdsOnly=true&f=json".format(where)
    j = urllib.request.urlopen(url_string)
    js = json.load(j)
    id_field = js["objectIdFieldName"]
    id_list = js["objectIds"]
    id_list.sort()
    num_of_records = len(id_list)
    print(("Number of target records: %s" % num_of_records))
    
    print("Gathering records…")
    features_list = []
    
    def load_features(urlstring, return_dict):
        succeed = False
        while not succeed:
            try:
                resp = requests.get(urlstring, verify=False)
                data = resp.json()
                gdf = gpd.GeoDataFrame.from_features(data['features'], crs='EPSG:4269')
                gdf = gdf.loc[gdf['geometry'].is_valid, :]
                return_dict[urlstring] = gdf
                succeed = True
            except:
                print ('Failed to load {}'.format(urlstring))
    processes = []
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    request_number = 0
    for i in range(0, num_of_records, max_records_count):
        request_number += 1
        to_rec = i + (max_records_count - 1)
        if to_rec > num_of_records:
            to_rec = num_of_records - 1
        from_id = id_list[i]
        to_id = id_list[to_rec]
        where = "{} >= {} and {} <= {}".format(id_field, from_id, id_field, to_id)
        print("  {}: {}".format(request_number, where))
        url_string = base_url + "/query?where={}&returnGeometry=true&outFields={}&f=geojson".format(where, fields)
    
        p = multiprocessing.Process(target=load_features, args=[url_string, return_dict])
        p.start()
        processes.append(p)
    
    for p in processes:
        p.join()
    p.close()
    
    for url in return_dict.keys():
        features_list.append(return_dict[url])
    treats = pd.concat(features_list)
    treats['activity_end'] = [datetime.datetime.fromtimestamp(ms/1000.0) for ms in treats['activity_end']]
    treats = treats.set_index('globalid')
    treats = treats.to_crs(32614)
    print ('Concatenating finished');
    end = time.perf_counter()
    print('-' * 50)
    print('Number of requests: {}'.format(request_number))
    print(f'Finished in {round(end - start, 2)} second(s)')
    return treats

In [31]:
TREAT_URLS = ["https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/0", 
              #"https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/1", 
              "https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/2"]
add_buffer = [True, True, False]
treats = [fetch_all_features(url) for url in TREAT_URLS]
treats.insert(1,gpd.GeoDataFrame())
treats = [{'table':treats[i], 'add_buffer':add_buffer[i]} for i in range(len(treats))]

handling https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/0 
Record extract limit: 2000
Number of target records: 807
Gathering records…
  1: objectid >= 1 and objectid <= 800
  2: objectid >= 801 and objectid <= 807
Concatenating finished
--------------------------------------------------
Number of requests: 2
Finished in 1.83 second(s)
handling https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/2 
Record extract limit: 2000
Number of target records: 8995
Gathering records…
  1: objectid >= 1 and objectid <= 800
  2: objectid >= 801 and objectid <= 1600
  3: objectid >= 1601 and objectid <= 2400
  4: objectid >= 2401 and objectid <= 3200
  5: objectid >= 3201 and objectid <= 4000
  6: objectid >= 4001 and objectid <= 4800
  7: objectid >= 4801 and objectid <= 5600
  8: objectid >= 5601 and objectid <= 6400
  9: objectid >= 6401 and objectid <= 7200
  10: objectid >= 7201 and objectid <= 8000
  11

In [17]:
treats.insert(1,gpd.GeoDataFrame())

In [9]:
if not os.path.exists(TREAT_DIR): 
    try: 
        os.makedirs(TREAT_DIR)
        print(f"Directory '{TREAT_DIR}' created successfully.") 
    except OSError as e: 
        print(f"Error creating directory '{TREAT_DIR}': {e}") 
else: 
    print(f"Directory '{TREAT_DIR}' already exists.") 
treats = []
for i in range(3):
    start=time.perf_counter()
    treat_fp = TREAT_DIR + TREAT_FPS[i]
    if not os.path.exists(treat_fp): 
        treat = fetch_all_features(TREAT_URLS[i])
        try: 
            treat.to_file(treat_fp)
            print(f"\n'{treat_fp}' created successfully.") 
        except e: 
            print(f"Error creating '{treat_fp}': {e}") 
    else: 
        print(f"\n'{treat_fp}' already exists. \n    Reading data...") 
        treat = gpd.read_file(treat_fp)
    end=time.perf_counter()
    print(f'    Finished in {round(end - start, 2)} second(s)')
    treats.append(treat)

Directory './data/treatment/' already exists.

'./data/treatment/point.geojson' already exists. 
    Reading data...
    Read data from './data/treatment/point.geojson'.
    Finished in 0.35 second(s)

'./data/treatment/line.geojson' already exists. 
    Reading data...


KeyboardInterrupt: 

# Work With the Data

In [ ]:
def get_num_intersections(objectid, buffer_distance):
    fire = fires.loc[objectid]
    alarm_date = fire['alarm_date']
    fire_poly = ogr.CreateGeometryFromWkt(str(fire['geometry']))
    
    poly_treats = treats[2]
    lp_treats = pd.concat(treats[:2])
    if fires.crs != poly_treats.crs:
        poly_treats = poly_treats.to_crs(fires.crs)
    if fires.crs != lp_treats.crs:
        lp_treats = lp_treats.to_crs(fires.crs)

    num_intersections = 0
    
    print("finding intersections with polygon treatment areas...")
    for idx_t, treat_row in poly_treats.iterrows():
        if (treat_row['activity_end'] < alarm_date):
            treat_poly = ogr.CreateGeometryFromWkt(treat_row['geometry'].wkt)
            if treat_poly.Intersects(fire_poly):
                num_intersections += 1
    print("finding intersections with point/line treatment areas...")       
    for idx_t, treat_row in lp_treats.iterrows():
        if (treat_row['activity_end'] < alarm_date):
            treat_thing = ogr.CreateGeometryFromWkt(treat_row['geometry'].wkt)
            treat_geom = treat_thing.Buffer(buffer_distance)
            if treat_geom.Intersects(fire_poly):
                num_intersections += 1
    print(num_intersections, "intersections")
    return num_intersections

In [ ]:
def get_poly_treatment_intersections(objectid):
    fire = fires.loc[objectid]
    alarm_date = fire['alarm_date']
    fire_poly = ogr.CreateGeometryFromWkt(str(fire['geometry']))
    poly_treats = treats[2]
    if fires.crs != poly_treats.crs:
        poly_treats = poly_treats.to_crs(fires.crs)
    #find treatments before fire that intersect with fire
    print("finding relevant treatment areas...")
    intersection_idxs = []
    for idx_t, treat_row in poly_treats.iterrows():
        if (treat_row['activity_end'] < alarm_date):
            treat_poly = ogr.CreateGeometryFromWkt(treat_row['geometry'].wkt)
            if treat_poly.Intersects(fire_poly):
                intersection_idxs.append(idx_t)
    #get intersections
    print("found {0} intersections".format(len(intersection_idxs)))
    print("finding areas of intersection...")
    poly_treats_relevant = poly_treats.loc[intersection_idxs]
    intersections = []
    for idx_t, treat_row in poly_treats_relevant.iterrows():
        treat_poly = ogr.CreateGeometryFromWkt(treat_row['geometry'].wkt)
        intersections.append(treat_poly.Intersection(fire_poly))
    return intersections

def get_line_treatment_intersections(objectid, buffer_distance):
    fire = fires.loc[objectid]
    alarm_date = fire['alarm_date']
    fire_poly = ogr.CreateGeometryFromWkt(fire['geometry'].wkt)
    lp_treats = pd.concat(treats[:2])
    if fires.crs != lp_treats.crs:
        lp_treats = lp_treats.to_crs(fires.crs)
    #find treatments before fire that intersect with fire
    print("finding relevant treatment areas...")
    intersection_idxs = []
    for idx_t, treat_row in lp_treats.iterrows():
        if (treat_row['activity_end'] < alarm_date):
            treat_thing = ogr.CreateGeometryFromWkt(treat_row['geometry'].wkt)
            treat_geom = treat_thing.Buffer(buffer_distance)
            if treat_geom.Intersects(fire_poly):
                intersection_idxs.append(idx_t)
    print("found {0} intersections".format(len(intersection_idxs)))
    #get intersections
    print("finding areas of intersection...")
    lp_treats_relevant = lp_treats.loc[intersection_idxs]
    intersections = []
    for idx_t, treat_row in lp_treats_relevant.iterrows():
        treat_thing = ogr.CreateGeometryFromWkt(str(treat_row['geometry']))
        treat_geom = treat_thing.Buffer(buffer_distance)
        intersections.append(treat_geom.Intersection(fire_poly))
    return intersections

In [45]:
def get_treatment_intersections(objectid, buffer_distance):
    fire = fires.loc[objectid]
    alarm_date = fire['alarm_date']
    fire_poly = ogr.CreateGeometryFromWkt(fire['geometry'].wkt)
    
    #this might be useful for parallelism
    fire_intersects = lambda geom: fire_poly.Intersects(geom)

    #look at line/point data
    lp_treats = pd.concat(treats[:2])
    
    intersection_ids = []
    intersection_geoms = []
    num_intersections = 0
    for treat_id, treat_row in lp_treats.iterrows():
        if (treat_row['activity_end'] < alarm_date):
            treat_geom = ogr.CreateGeometryFromWkt(treat_row['geometry'].wkt)
            treat_geom = treat_geom.Buffer(buffer_distance)
            
            if treat_geom.Intersects(fire_poly):
                intersection_ids.append(treat_id)
                intersection_geoms.append(treat_geom)
                num_intersections += 1
    lp_data = [[objectid, intersection_ids[i], intersection_geoms[i].Intersection(fire_poly)] for i in range(num_intersections)]

    #look at polygon data
    poly_treats = treats[2]
    if fires.crs != poly_treats.crs:
        poly_treats = poly_treats.to_crs(fires.crs)
    intersection_ids = []
    intersection_geoms = []
    num_intersections = 0
    for treat_id, treat_row in poly_treats.iterrows():
        if (treat_row['activity_end'] < alarm_date):
            treat_poly = ogr.CreateGeometryFromWkt(treat_row['geometry'].wkt)
            if treat_poly.Intersects(fire_poly):
                intersection_idxs.append(treat_id)
                intersection_geoms.append(treat_poly)
                num_intersctions += 1
    poly_data = [[objectid, intersection_ids[i], intersection_geoms[i].Intersection(fire_poly)] for i in range(num_intersections)]
    all_intersection_data = lp_data.extend(poly_data)
    intersection_df = gpd.GeoDataFrame(all_intersection_data, columns=['fire_objectid', 'treat_globalid', 'intersection'])
    return intersection_df

In [46]:
get_treatment_intersections(fires.index[0], 20)

KeyboardInterrupt: 

In [16]:
fires.loc[objectid]['alarm_date']

Timestamp('2022-05-24 00:00:00')

In [25]:
from time import sleep

In [47]:
def get_feature_intersections(objectid, fire_poly, treatments, add_buffer, buffer_distance=20):
    intersection_ids = []
    intersection_geoms = []
    num_intersections = 0
    for treat_id, treat_row in treats_valid_date.iterrows():
        treat_poly = ogr.CreateGeometryFromWkt(treat_row['geometry'].wkt)
        if add_buffer:
            treat_poly = treat_poly.Buffer(buffer_distance)
        if treat_poly.Intersects(fire_poly):
            intersection_ids.append(treat_id)
            intersection_geoms.append(treat_poly)
            num_intersections += 1
    poly_data = [[objectid, intersection_ids[i], intersection_geoms[i].Intersection(fire_poly)] for i in range(num_intersections)]
    int_df = gpd.GeoDataFrame(int_data, columns=['fire_objectid', 'treat_globalid', 'intersection'])
    return int_df
    
def get_all_intersections(objectid):
    fire = fires.loc[objectid]
    alarm_date = fire['alarm_date']
    fire_poly = ogr.CreateGeometryFromWkt(fire['geometry'].wkt)
    fire_intersects = lambda geom: fire_poly.Intersects(geom)
    int_dfs = []
    for item in treats:
        feature = item['table']
        print(type(feature))
        print(feature.columns)
        print(feature['activity_end'] < alarm_date)
        idxs = feature['activity_end'] < alarm_date
        treats_valid_date = feature[idxs]
        add_buffer = item['add_buffer']
        int_dfs.append(get_feature_intersections(objectid, fire_poly, treats_valid_date, add_buffer))
    return gpd.concatenate(int_dfs)

In [48]:
objectid = fires.index[6]
get_all_intersections(objectid)

<class 'geopandas.geodataframe.GeoDataFrame'>
Index(['geometry', 'agency', 'county', 'activity_description',
       'activity_quantity', 'broad_vegetation_type', 'activity_end',
       'entity_type', 'administering_org', 'primary_ownership_group',
       'activity_uom', 'activity_cat', 'region', 'objectid',
       'activity_status'],
      dtype='object')
globalid
{5CB5D324-ADC4-4533-8C3E-21C708858EE0}     True
{04E086B0-5A46-4F6C-9BE4-61A7020AE4A4}     True
{3951AE08-91D6-4781-8956-64459F20C442}     True
{11B106B0-B047-47A1-B693-501028652B51}     True
{770EC70F-1B22-4F9E-9B9D-038FA7C29F70}     True
                                          ...  
{075B887E-6532-4F96-AC89-6B6F4F4C076D}     True
{63BE246A-3920-4FAB-8897-EDAFDB3C85BF}     True
{36B8AECA-2114-4C8B-BDBB-DC36934D3D27}     True
{162B3CE7-0B51-48B3-B1CD-59FA816AEEE0}     True
{60E04E98-25D7-4BC0-94F6-768D9A5CDBC7}    False
Name: activity_end, Length: 807, dtype: bool
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex(sta

KeyError: 'activity_end'

In [28]:
objectid = fires.index[6]
buffer_distance=20
add_buffer=True

fire = fires.loc[objectid]
alarm_date = fire['alarm_date']
fire_poly = ogr.CreateGeometryFromWkt(fire['geometry'].wkt)

#this might be useful for parallelism
fire_intersects = lambda geom: fire_poly.Intersects(geom)
            
#look at line/point data
lp_treats = pd.concat(treats[:2])
intersection_ids = []
intersection_geoms = []
num_intersections = 0
treats_valid_date = lp_treats[lp_treats['activity_end'] < alarm_date]
for treat_id, treat_row in treats_valid_date.iterrows():
    if (treat_row.loc['activity_end'] < alarm_date):
        treat_poly = ogr.CreateGeometryFromWkt(treat_row['geometry'].wkt)
        if (add_buffer):
            treat_poly = treat_poly.Buffer(buffer_distance)
        if treat_geom.Intersects(fire_poly):
            intersection_ids.append(treat_id)
            intersection_geoms.append(treat_geom)
            num_intersections += 1
            print(num_intersections)
print(num_intersections, "intersections with point data")
int_data = [[objectid, intersection_ids[i], intersection_geoms[i].Intersection(fire_poly)] for i in range(num_intersections)]
int_df = gpd.GeoDataFrame(int_data, columns=['fire_objectid', 'treat_globalid', 'intersection'])
int_df

0 intersections with point data


,fire_objectid,treat_globalid,intersection
